# Yolo-v2-tiny DW2TF validation

In [1]:
import numpy as np
import tensorflow as tf

# Random input data to test 
in_data = np.random.randn(64, 416, 416, 3) * 120.56

### Output from Darkflow converted Yolo-v2-tiny

In [2]:
tf.reset_default_graph()

saver = tf.train.import_meta_graph('/data/darkflow/yolov2-tiny.ckpt.meta')
ckpt_path = '/data/darkflow/yolov2-tiny.ckpt'

g = tf.get_default_graph()
in_t = g.get_tensor_by_name('input:0')
out_t = g.get_tensor_by_name('output:0')

with tf.Session() as sess:
    saver.restore(sess, ckpt_path)

    out_data_darkflow = sess.run(out_t, feed_dict={in_t: in_data})

W0730 11:43:31.104432 140471296845568 deprecation.py:323] From /scratch/anaconda3/envs/tf1.14/lib/python3.6/site-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


### Output from DW2TF converted Yolo-v2-tiny

In [3]:
tf.reset_default_graph()

saver = tf.train.import_meta_graph('/data/DW2TF/data/yolov2-tiny.ckpt.meta')
ckpt_path = '/data/DW2TF/data/yolov2-tiny.ckpt'

g = tf.get_default_graph()
in_t = g.get_tensor_by_name('yolov2-tiny/net1:0')
out_t = g.get_tensor_by_name('yolov2-tiny/convolutional9/BiasAdd:0')

with tf.Session() as sess:
    saver.restore(sess, ckpt_path)
    
    out_data_dw2tf = sess.run(out_t, feed_dict={in_t: in_data})

### RMS Error

In [4]:
print(out_data_darkflow.shape)
print(out_data_dw2tf.shape)
print(np.mean(out_data_darkflow - out_data_dw2tf)**2)

(64, 13, 13, 425)
(64, 13, 13, 425)
7.609460173061993e-14


In [5]:
print(out_data_darkflow[0,0,0,:10])

[ -42.268208   13.246095    8.282185  -11.273804 -104.089836  144.73775
   52.463158  150.64749    68.77003    16.30984 ]


In [6]:
print(out_data_dw2tf[0,0,0,:10])

[ -42.268047   13.246207    8.28245   -11.273606 -104.08937   144.7373
   52.461124  150.6482     68.76962    16.310291]
